# app citation에 grant dummy 붙이기 

### merge 관련 issues  
1. 분석 범위: 2002년 ~ 2013년  
=> app_cite, app_granted 모두 2002 ~ 2013에 해당하는 친구들만 필요
2. 2002 ~ 2013에 해당하는 app은 app_profile에 있음
3. app_cite data가 아직 없어서 app_id 썼는지 DN 썼는지 모름  
=> 그렇다면 일단 app_profile에 granted dummy를 붙이자
4. 하지만 app_granted는 app_id, DN 섞여있음  
=> 2개 key (app_id, DN) 각각 join 후, column을 하나로 합침

### 1. app_profile + granted dummy

In [1]:
import csv
import os
import pandas as pd
import numpy as np
import math

os.chdir('E:/apps')

In [2]:
grt = pd.read_csv('app_granted.csv')
grt.iloc[0:5,:]

C:\Users\DW\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,number,app_id,granted_dum
0,NaN,07/683484,0.0
1,NaN,08/537802,0.0
2,NaN,08/541191,0.0
3,NaN,08/565943,0.0
4,NaN,08/579733,0.0


__문제__
- granted_dum에는 0, 1, NaN이 섞여 있음   
=> Join 시 생기는 NaN과 원래 있었던 NaN 구분 불가 
- 그렇다면 원래 있는 NaN을 2로 바꿔버리자!

In [3]:
grt['granted_dum'].fillna(2, inplace=True)
grt.loc[grt['number'] == 20050000001]

,number,app_id,granted_dum
1044847,2.005000e+10,NaN,2.0


In [4]:
grt_id = grt.iloc[:,1:3]
grt_id.rename(columns={'granted_dum': 'granted_dum_i'}, inplace=True)

grt_n = grt.iloc[:,[0,2]]
grt_n.rename(columns={'granted_dum': 'granted_dum_n'}, inplace=True)

print list(grt_id), list(grt_n)

['app_id', 'granted_dum_i'] ['number', 'granted_dum_n']


C:\Users\DW\Anaconda2\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [5]:
del grt

prf = pd.read_csv('app_profile_melted.csv')

prf = prf.merge(grt_id, how='left')
prf = prf.merge(grt_n, how='left')

merge 후 granted_dum_i, granted_dum_i 중 __하나만 NaN 일 때__ 정상  
둘 다 NaN일 경우 (matching 실패) or 둘 다 NaN 아닐 경우 (matching 중복) 처리 필요  
=> numpy array로 처리 후 다시 갖다 붙이자

In [18]:
prf_dum = prf.iloc[:,5:7]
prf.to_csv('app_prf_temp.csv',index=False)
prf_dum.to_csv('grtdum_temp.csv', index=False)

In [2]:
# dummy 뒷처리

def granted_dum(arr):
    a = arr[0] + arr[1]
    if math.isnan(a):
        if not math.isnan(arr[0]):
            a = arr[0]
        else:
            a = arr[1] if not math.isnan(arr[1]) else a
            a = np.nan if a==2 else a
     
    else:
        if a==2:
            if arr[0]==arr[1]:
                a=1
            
            else:
                a=0
        
        elif a==3:
            a=1
            
    return a

In [3]:
dum = np.genfromtxt('grtdum_temp.csv',delimiter=',',names=True)

vfunc = np.vectorize(granted_dum)
result = vfunc(dum)
np.savetxt('grtdum.csv', result,delimiter=',', fmt='%1.1f', header='granted_dum', comments='')

In [4]:
prf = pd.read_csv('app_prf_temp.csv')
dum = pd.read_csv('grtdum.csv')

prf = prf.iloc[:,0:5]

In [6]:
print len(prf), len(dum)

3642191 3642191


In [8]:
result = pd.concat([prf, dum], axis=1)
result.iloc[0:5,:]

,number,app_id,date,assignee,country,granted_dum
0,20020000001,09/836917,2001-04-18,private,NaN,1.0
1,20020000002,09/888723,2001-06-25,private,NaN,1.0
2,20020000003,09/894736,2001-06-28,private,NaN,1.0
3,20020000004,09/939364,2001-08-24,private,NaN,1.0
4,20020000005,09/552169,2000-04-18,private,NaN,1.0


In [10]:
result.to_csv('app_profile_granted.csv',index=False)